In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
data = Path("../data")
plot_path = data / "plots"
map_path = data / "graph3_map"
stat_path = map_path / "stats"
geojson_path = map_path / "geojson"

import json

# Load geojson data


In [2]:
# Load custom.geo.json
with open(geojson_path / "custom.geo.json") as f:
    geojson = json.load(f)


# Each of the feature in "features" jas a "properties" fields, and inside of this field an "admin" fields. Put all these in a dataframe
df_geojson = pd.DataFrame(
    [
        {
            "country": feature["properties"]["admin"],
        }
        for feature in geojson["features"]
    ]
)
df_geojson["country"].sort_values()

89        Afghanistan
15            Albania
156           Algeria
152            Angola
41          Argentina
            ...      
132    Western Sahara
93              Yemen
139            Zambia
153          Zimbabwe
158          eSwatini
Name: country, Length: 179, dtype: object

In [3]:
with open(geojson_path / "countries.geo.json") as f:
    countries_geo_json = json.load(f)

df_countries_geo_json = pd.DataFrame(
    [
        {
            "country": feature["properties"]["name"],
        }
        for feature in countries_geo_json["features"]
    ]
)
df_countries_geo_json["country"].sort_values()

0         Afghanistan
2             Albania
45            Algeria
1              Angola
6          Antarctica
            ...      
172         West Bank
136    Western Sahara
173             Yemen
175            Zambia
176          Zimbabwe
Name: country, Length: 177, dtype: object

In [ ]:
with open(geojson_path / "countries.geojson") as f:
    countries_geojson = json.load(f)

In [ ]:
geojson = pd.read_json(data / "map/custom.geo.json")
geojson

# All the "admin" fields
["country"].sort_values()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'featurecla..."
1,FeatureCollection,"{'type': 'Feature', 'properties': {'featurecla..."
2,FeatureCollection,"{'type': 'Feature', 'properties': {'featurecla..."
3,FeatureCollection,"{'type': 'Feature', 'properties': {'featurecla..."
4,FeatureCollection,"{'type': 'Feature', 'properties': {'featurecla..."
...,...,...
174,FeatureCollection,"{'type': 'Feature', 'properties': {'featurecla..."
175,FeatureCollection,"{'type': 'Feature', 'properties': {'sovereignt..."
176,FeatureCollection,"{'type': 'Feature', 'properties': {'sovereignt..."
177,FeatureCollection,"{'type': 'Feature', 'properties': {'sovereignt..."
